In [45]:
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
from datetime import date as dt

In [46]:
# set parameters for CAPM. Choose your security, benchmark, risk free rate (or proxy for risk free rate), start & end dates for the downloaded data
RISKY_ASSET = 'AXP'
MARKET_BENCHMARK = '^SP500TR' #S&P 500 total return
MKT_RF ='^IRX' #90 T-bill
START_DATE = '2013-03-31'
END_DATE = dt.today()

In [47]:
# create data frame of timeseries for asset, benchmark, and risk free rate proxy
df = yf.download([RISKY_ASSET, MARKET_BENCHMARK, MKT_RF],start=START_DATE,end=END_DATE,progress=False)
# calculate returns
df = df['Adj Close'].rename(columns={RISKY_ASSET: 'asset', MARKET_BENCHMARK: 'market', MKT_RF: 'mkt_rf'}) \
    .resample('M') \
    .last() \
    .pct_change() \
    .dropna()
df.iloc[-5:]

,asset,mkt_rf,market
Date,,,
2022-12-31,-0.062440,0.001646,-0.057615
2023-01-31,0.188063,0.072770,0.062834
2023-02-28,-0.005374,0.030635,-0.024400
2023-03-31,-0.051957,-0.026539,0.036714
2023-04-30,-0.006822,0.069357,0.007528


In [48]:
rf_series = df.loc[:,'mkt_rf'] # 90 T-bill return series 
df = df.subtract(rf_series, axis=0) # subtract RF return series from asset and benchmark series
df= df.drop(['mkt_rf'], axis=1) # Drop RF series (now all 0 values)
df.iloc[-5:]

,asset,market
Date,,
2022-12-31,-0.064086,-0.059261
2023-01-31,0.115293,-0.009936
2023-02-28,-0.036008,-0.055034
2023-03-31,-0.025418,0.063254
2023-04-30,-0.076179,-0.061829


In [49]:
# covariance method for beta calculation (Cov(asset, benchmark) divided by volatility of market returns)
covariance = df.cov().iloc[0,1]
benchmark_variance = df.market.var()
beta = covariance / benchmark_variance
beta

1.0020084417869242

In [50]:
# CAPM regression setup
y = df.pop('asset') #extract dependent variable (asset) times series
x = sm.add_constant(df) # independent variable (benchmark) time series
capm_model = sm.OLS(y, x).fit() # Ordinary Least Squares regression from statsmodels
print(capm_model.summary()) # Regression output. market coefficient should equal beta variable above from covariance method


                            OLS Regression Results                            
Dep. Variable:                  asset   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 1.114e+05
Date:                Sat, 15 Apr 2023   Prob (F-statistic):          2.02e-177
Time:                        01:27:31   Log-Likelihood:                 183.77
No. Observations:                 120   AIC:                            -363.5
Df Residuals:                     118   BIC:                            -358.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0015      0.005      0.292      0.7